Import des librairies

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pythermalcomfort import two_nodes
from SALib.analyze import rbd_fast
from SALib.sample import latin
from SALib.plotting.bar import plot as barplot


In [ ]:

current_directory = os.getcwd()

# List all files in the current directory
files = os.listdir(current_directory)

# Iterate through all files
for file in files:
    if file.endswith('.py'):
        # Skip Python files
        continue
    else:
        # Delete non-Python files
        os.remove(os.path.join(current_directory, file))

In [ ]:
model = "Gagge's Two-Nodes" # Name of model for plot titles
filename='GAGGE_2NODES' # Eventual filename for exporting all csv outputs
doyouwanttoploteverything = True  # Flag if will to export all data in csv and all plot for each degree step (True if will to plot)

In [ ]:
def model4sa(tdb,
                tr,
                v,
                rh,
                met,
                clo,
                standard="ASHRAE", limit_inputs=False, round=False):
    
    pythermal=two_nodes(tdb=tdb,
                        tr=tr,
                        v=v,
                        rh=rh,
                        met=met,
                        clo=clo,
                        standard="ASHRAE", limit_inputs=False, round=False)
    
    pts = 0.25 * pythermal['_set'] - 6.03
    es=1.33
    l=0.195
    tsv_se = es * pts
    tsv_sa = pts/(1+l*pts)
    
    return {"_set": pythermal['_set'],
                    "et":pythermal['et'],
                    "t_core" : pythermal['t_core'],
                    "t_skin" : pythermal['t_skin'],
                    "pmv_gagge" : pythermal['pmv_gagge'],
                    "pmv_set" : pythermal['pmv_set'],
                    'w' : pythermal['w'],
                    'w_max' : pythermal['w_max'],
                    "disc" : pythermal['disc'],
                    "t_sens" : pythermal['t_sens'],
                    'pts' : pts,
                    'tsv_se':tsv_se,
                    'tsv_sa':tsv_sa}

In [ ]:
model_output_names="_set"
                    # "et",
                    # "t_core",
                    # "t_skin",
                    # "pmv_gagge",
                    # "pmv_set",
                    # 'w',
                    # 'w_max',
                    # "disc",
                    # "t_sens",
                    # 'pts',
                    # 'tsv_se',
                    # 'tsv_sa'
                    
                    

In [ ]:
# Define number of sample for SA, need to be determined after a Si (S1) convergence study
samples=10000
# Define input variable ranges based on possible model inputs
lower_v = 0.01 
upper_v = 2.2
# lower_met = 0.65
# upper_met = 1.26
lower_clo = 0.29
upper_clo = 0.79
lower_tdb = 25.4
upper_tdb = 32.1
lower_tr = 23.1
upper_tr = 31.2
lower_rh = 60
upper_rh = 90

In [ ]:
# Define SA problem (SALIB package)
problem = {
    'num_vars': 5, # number of variable inputs
    'names': [ 'tdb','tr','rh','v','clo'], # input names in correct order
    'bounds': [
        [lower_tdb, upper_tdb],
        [lower_tr, upper_tr],
        [lower_rh, upper_rh],
        [lower_v, upper_v],
        [lower_clo, upper_clo]
            ],
    'dist' : ['unif',
              'unif',
              'unif',
              'unif',
              'unif']    
            
        }

In [ ]:
# Generate the parameter samples using SALIB RBDfast recommanded method (Latin Hypercube sampling)
sampling = latin.sample(problem, samples)
# Evaluate model
Y = [model4sa(tdb=params[0],
                    tr=params[1],
                    rh=params[2],
                    v=params[3],
                    met=1.1,
                    clo=params[4],
                    standard="ASHRAE", limit_inputs=False, round=False)[model_output_names] for params in sampling
                    ]

In [ ]:
pd.DataFrame(Y)

In [ ]:
Si = rbd_fast.analyze(problem, sampling, np.array(Y), print_to_console=True)

In [ ]:
# plt.title='Indice de Sobol du premier ordre'
Si_df=pd.DataFrame(Si)
print(Si_df)
names=Si['names']
ax=barplot(Si_df)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names)



In [ ]:
plot(problem, Si, threshold=0.05, names_latex=None)